In [27]:
import pandas as pd
import numpy as np
import keras
from keras.utils.data_utils import get_file
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from os.path import join
import multiprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from scipy.stats import skew, kurtosis, tvar, entropy, pearsonr

%matplotlib inline

In [2]:
DATA_HOME = '../lfw-deepfunneled-Cropped/'

In [3]:
PAIRS_PATH = '../pairs.txt'

In [4]:
raw_dataset = pd.read_csv(PAIRS_PATH, nrows=None)
raw_dataset.columns = ['data']

In [5]:
raw_dataset = raw_dataset

In [6]:
def split_raw_data(row):
    fields = str(row[0]).split('\t')
    is_the_same = 0
    path1, path2 = '{}/{}_{:04d}.jpg', '{}/{}_{:04d}.jpg'
    if len(fields) == 3:
        is_the_same = 1
        path1 = path1.format(fields[0], fields[0], int(fields[1]))
        path2 = path2.format(fields[0], fields[0], int(fields[2]))
    elif len(fields) == 4:
        path1 = path1.format(fields[0], fields[0], int(fields[1]))
        path2 = path2.format(fields[2], fields[2], int(fields[3]))
    else:
        raise(Exception('invalid fields'))
    
    return pd.Series([path1, path2, is_the_same], ['path1', 'path2', 'is_the_same']) 

In [7]:
dataset = raw_dataset.apply(split_raw_data, axis=1)

In [8]:
dataset.head()

,path1,path2,is_the_same
0,Abel_Pacheco/Abel_Pacheco_0001.jpg,Abel_Pacheco/Abel_Pacheco_0004.jpg,1
1,Akhmed_Zakayev/Akhmed_Zakayev_0001.jpg,Akhmed_Zakayev/Akhmed_Zakayev_0003.jpg,1
2,Akhmed_Zakayev/Akhmed_Zakayev_0002.jpg,Akhmed_Zakayev/Akhmed_Zakayev_0003.jpg,1
3,Amber_Tamblyn/Amber_Tamblyn_0001.jpg,Amber_Tamblyn/Amber_Tamblyn_0002.jpg,1
4,Anders_Fogh_Rasmussen/Anders_Fogh_Rasmussen_00...,Anders_Fogh_Rasmussen/Anders_Fogh_Rasmussen_00...,1


In [9]:
def path2ImgVec(path):
    x = img_to_array(load_img(join(DATA_HOME, path)))
    return x.reshape((1,) + x.shape)

In [10]:
img_paths = list(set(dataset.path1.tolist()+dataset.path2.tolist()))
len(img_paths)

7701

In [11]:
pool = multiprocessing.Pool(8)
results = pool.map(path2ImgVec, img_paths)
pool.close()
pool.join()

In [12]:
img_vecs = np.vstack(results)
img_vecs.shape

(7701, 55, 47, 3)

In [13]:
input_shape = img_vecs.shape[1:]
img_vecs.shape

(7701, 55, 47, 3)

In [14]:
from keras.models import load_model

model = load_model('../models/webface-simple-cnn.3348.model.h5')

In [15]:
from keras import backend as K
inp = model.input
# outputs = [layer.output for layer in model.layers]
outputs = [model.layers[-2].output]
functor = K.function([inp] + [K.learning_phase()], outputs)

In [16]:
deepids = functor([img_vecs, 1.])[0]

In [17]:
deepids.shape

(7701, 160)

In [18]:
img_deepid_mem = {k:v for k, v in zip(img_paths, deepids)}

In [92]:
from scipy.spatial.distance import cosine, cityblock, jaccard, canberra, euclidean, minkowski, braycurtis

COL_NAMES = [
    "z_cosine",
#     'z_cityblock',
#     'z_jaccard',
#     "z_canberra",
#     "z_euclidean",
#     "z_minkowski",
#     "z_braycurtis",
    'z_skew1',
    'z_skew2',
    'z_kurtosis1',
    'z_kurtosis2',
#     'z_tvar1',
#     'z_tvar2',
] + ['is_the_same']

def to_deepid_features(row):
    vec1 = img_deepid_mem[row['path1']]
    vec2 = img_deepid_mem[row['path2']]

    feats = [
        cosine(vec1, vec2),
#         cityblock(vec1, vec2),
#         jaccard(vec1, vec2),
#         canberra(vec1, vec2),
#         euclidean(vec1, vec2),
#         minkowski(vec1, vec2, 3),
#         braycurtis(vec1, vec2),
        skew(vec1),
        skew(vec2),
        kurtosis(vec1),
        kurtosis(vec2),
#         tvar(vec1),
#         tvar(vec2),
    ]
    
    return pd.Series(feats + [row['is_the_same']], COL_NAMES) 

In [93]:
feat_dataset = dataset.apply(to_deepid_features, axis=1)

In [94]:
feat_dataset.head()

,z_cosine,z_skew1,z_skew2,z_kurtosis1,z_kurtosis2,is_the_same
0,0.456116,1.045443,0.860761,0.665772,-0.308264,1.0
1,0.225153,1.280119,0.874431,0.713357,-0.060878,1.0
2,0.370649,0.900829,0.874431,-0.064136,-0.060878,1.0
3,0.280739,1.148976,1.127408,1.116505,0.397695,1.0
4,0.437203,0.981824,1.016176,-0.026174,0.260809,1.0


In [95]:
col = [c for c in feat_dataset.columns if c[0]=='z']

X = feat_dataset[col].as_matrix()
y = feat_dataset['is_the_same'].as_matrix()

In [96]:
X[0]

array([ 0.45611648,  1.04544306,  0.86076117,  0.66577224, -0.30826415])

In [103]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [104]:
import xgboost as xgb
params = {}
params["objective"] = "binary:logistic"
params['eval_metric'] = ['logloss', 'error']
# params["eta"] = 0.02
# params["subsample"] = 0.7
# params["min_child_weight"] = 1
# params["colsample_bytree"] = 0.7
# params["max_depth"] = 4
# params["silent"] = 1
params["seed"] = 1632

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]
bst = xgb.train(params, d_train, 500, watchlist, early_stopping_rounds=50, verbose_eval=100)

[0]	train-logloss:0.597146	train-error:0.222083	valid-logloss:0.613671	valid-error:0.265
Multiple eval metrics have been passed: 'valid-error' will be used for early stopping.

Will train until valid-error hasn't improved in 50 rounds.
Stopping. Best iteration:
[8]	train-logloss:0.438248	train-error:0.206458	valid-logloss:0.516222	valid-error:0.2525

